In [5]:
import pandas as pd
import numpy as np

In [4]:
filename='./df.B1_1.csv'
pd=pd.read_csv(filename)

In [5]:
pd.head()

,RelativeTime,PacketsSent,PacketsReceived,BytesSent,BytesReceived,IPSrc0,IPDst0,Protocol0,PacketsSent0,PacketsReceived0,...,CollectData,EpochTime,StartTime,PlaybackProgress,Length,BufferHealth,BufferProgress,BufferValid,KBytesReceived,label
0,0.2,21,19,7263,14866,192.168.1.15,173.194.185.106,0,0,0,...,0,0,1533532916170,0.000000,668.101,0.000000,0.000000,-1,14.866,NoQualityInfo
1,0.9,10,18,688,21888,192.168.1.15,173.194.185.106,0,0,0,...,0,0,1533532916170,0.000000,668.101,0.000000,0.000000,-1,21.888,NoQualityInfo
2,0.4,7,16,490,16757,192.168.1.15,173.194.131.41,0,0,0,...,0,0,1533498438497,0.000000,617.801,0.000000,0.000000,-1,16.757,NoQualityInfo
3,1.0,10,15,1017,16544,192.168.1.15,173.194.61.41,0,0,0,...,0,0,1533491556102,0.000000,617.801,0.000000,0.000000,-1,16.544,NoQualityInfo
4,4.9,38,320,3977,439231,192.168.1.15,173.194.61.41,QUIC,34,316,...,0,1533491560158,1533491556102,0.017419,617.801,10.117581,0.016405,true,439.231,q1080p


# Chunk Detection

In [27]:
from scapy.all import *

GET_THRESH = 300 # bytes
DOWN_THRESH = 300  # bytes
NETINFO_NUM = 25

audio=[]
video=[]

def isUplink(p):
    return p[IP].src.startswith('192.168.')

def detectAV(c):
    if c[1] < 680:
        flag=0
    else:
        flag=1
    if c[4]<=80*1024:
        flag=2
    return flag

def chunkDetection(filename):
    first = True
    a = rdpcap(filename)
    meta_time = float(a[0].time)
    for p in a:
        if p.haslayer(IP):
            pLen=p[IP].len
            ip_time=float(p.time)
            if isUplink(p) and pLen > GET_THRESH:
                if not first:
                    c = [timestamp, getSize, downStart, downEnd, chunkSize, protocol]
                    avFlag=detectAV(c)
                    if avFlag==0:
                        audio.append(c)
                    elif avFlag==1:
                        video.append(c)
                    else:
                        continue
                timestamp = ip_time
                getSize = pLen
                chunkSize = 0
                downFlag = False
            elif not isUplink(p) and pLen > DOWN_THRESH:
                first = False
                if not downFlag:
                    downStart = ip_time
                    downFlag = True
                downEnd = ip_time
                chunkSize += pLen
                protocol = p[IP].proto
    return meta_time

filename = 'RequetDataSet-master/A/PCAP_FILES/baseline_Jan17_exp_28.pcap'
meta_time=chunkDetection(filename)

In [28]:
gmtime(meta_time)

time.struct_time(tm_year=2018, tm_mon=1, tm_mday=23, tm_hour=20, tm_min=19, tm_sec=2, tm_wday=1, tm_yday=23, tm_isdst=0)

In [29]:
audio

[[1516738742.655203, 380, 1516738742.700483, 1516738744.314245, 552327, 6],
 [1516738744.377967, 516, 1516738744.4634, 1516738746.505036, 106481, 6],
 [1516738746.873918, 310, 1516738746.935163, 1516738748.219621, 564971, 6],
 [1516738749.692165, 367, 1516738749.735744, 1516738751.794369, 1920180, 6],
 [1516738765.211107, 615, 1516738769.842657, 1516738775.184342, 487232, 6],
 [1516738775.213342, 616, 1516738775.224999, 1516738782.011702, 1479360, 6],
 [1516738784.207095, 617, 1516738784.502093, 1516738789.040908, 260544, 6],
 [1516738834.691745, 569, 1516738834.710337, 1516738837.718746, 278208, 6],
 [1516738837.764984, 319, 1516738837.825496, 1516738844.080981, 369099, 6],
 [1516738864.205045, 640, 1516738864.223467, 1516738869.897777, 1605805, 6],
 [1516738869.949259, 360, 1516738870.195757, 1516738873.358352, 603520, 6],
 [1516738875.206717, 623, 1516738875.553429, 1516738892.04944, 1203004, 6],
 [1516738892.247305, 397, 1516738892.345109, 1516738919.911091, 104512, 6],
 [151673892

In [30]:
video

[[1516738746.508449, 1240, 1516738746.520845, 1516738746.87212, 109638, 6],
 [1516738748.221699, 1362, 1516738748.222527, 1516738748.551708, 384432, 6],
 [1516738748.567663, 1277, 1516738748.793789, 1516738748.85303, 95582, 6],
 [1516738748.86137, 1366, 1516738748.861926, 1516738749.297839, 615198, 6],
 [1516738749.310627, 1279, 1516738749.331315, 1516738749.558124, 127483, 6],
 [1516738749.570114, 1368, 1516738749.5976, 1516738749.689056, 88705, 6],
 [1516738751.80852, 1279, 1516738751.914414, 1516738752.470491, 293622, 6],
 [1516738753.86552, 784, 1516738754.153523, 1516738763.12392, 2139095, 6],
 [1516738763.127886, 1280, 1516738763.137053, 1516738763.729181, 458502, 6],
 [1516738782.029833, 1281, 1516738782.100171, 1516738784.127425, 312949, 6],
 [1516738789.048652, 1369, 1516738789.050118, 1516738797.467386, 121833, 6],
 [1516738799.09586, 1197, 1516738811.400853, 1516738825.540612, 90361, 6],
 [1516738825.57264, 1240, 1516738825.612653, 1516738834.68308, 96808, 6],
 [1516738844.1

In [36]:
columns = ['start_time', 'type', 'ttfb', 'download_time', 'chunk_size', 'rel_start_time', 'rel_end_time']
df_chunk=pd.DataFrame(columns=columns)

for v in video:
    start_time = v[0]
    type = 'v'
    ttfb = v[2]-start_time
    download_time = v[3]-v[2]
    chunk_size = v[4]
    rel_start_time = start_time-meta_time
    rel_end_time = v[3]-meta_time
    s = pd.Series([start_time, type, ttfb, download_time, chunk_size, rel_start_time, rel_end_time], index=columns)
    df_chunk.loc[len(df_chunk)] = s

for a in audio:
    start_time = a[0]
    type = 'a'
    ttfb = a[2]-start_time
    download_time = a[3]-a[2]
    chunk_size = a[4]
    rel_start_time = start_time-meta_time
    rel_end_time = a[3]-meta_time
    s = pd.Series([start_time, type, ttfb, download_time, chunk_size, rel_start_time, rel_end_time], index=columns)
    df_chunk.loc[len(df_chunk)] = s

df_chunk.sort_values(by='start_time', inplace=True)
df_chunk.to_csv('results/chunk_detection/A/baseline_Jan17_exp_28.csv', index=False)